# Imports

In [59]:
from google.cloud import storage
import pandas as pd
import numpy as np
import re
from unidecode import unidecode
import ast

## Funções auxiliares

In [51]:
def get_data(bucket_name:str, imobiliarias:str = ['apolar', 'cilar'],by:str = ['date','date_diff'], dates:list = [], date_diff:int = 2):
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)

    files_on_bucket = [i.name for i in bucket.list_blobs()]
    files = pd.DataFrame(files_on_bucket, columns=['name'])
    files['date'] = pd.to_datetime(files['name'].apply(lambda f: f.split(' - ')[0]))
    files['imobiliaria'] = files['name'].apply(lambda f: f.split(' - ')[-1].replace('.csv',''))

    match by:
        case 'date':
            files = files.loc[files['date'].isin(dates)]
            files = files.loc[files['imobiliaria'].isin(imobiliarias)]
        case 'date_diff':
            dates = files['date'].sort_values(ascending=False).drop_duplicates().reset_index(drop=True)[:date_diff].tolist()
            files = files.loc[files['date'].isin(dates)]
            files = files.loc[files['imobiliaria'].isin(imobiliarias)]
    
    df_full = pd.DataFrame()

    for file_name in files['name'].tolist():
        try:
            df_aux = pd.read_csv(f'gs://{bucket_name}/{file_name}')
            df_full = pd.concat([df_full, df_aux], axis = 0)
        except:
            pass

    df_full = df_full.reset_index(drop=True)

    return df_full

# Carregando Dados

In [74]:
df_full = get_data(
    bucket_name='busca-apartamentos-bucket',
    imobiliarias=['cilar'],
    by='date',
    dates=['2024-03-17']
)

/tmp/ipykernel_922727/864656196.py:12: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  files = files.loc[files['date'].isin(dates)]


In [75]:
df = df_full.copy()

In [76]:
df['detalhes'] = df['detalhes'].apply(lambda x: ast.literal_eval(x))

In [78]:
df

,site,data_coleta,titulo,link,endereco,detalhes,aluguel,condominio,iptu,catacteristicas_imovel,detalhes_condominio,mais_detalhes_imovel
0,Cilar,2024-03-17,Apartamento no 15º andar com 3 quartos,https://cilar.com.br/alugar/apartamento-no-15-...,"Rua Professor Pedro Viriato Parigot Souza, 306...","[Características, do, imóvel, Área, Total, 421...","AluguelR$ 13.000,00","Condominio R$ 3.189,70","IPTU R$ 874,00",1ª LOCAÇÃO Apartamento de alto padrão contend...,"Piscina, Academia, Salão de festas, Playground...","Andar: 15, Ano de construção: 2021, Aptos por ..."
1,Cilar,2024-03-17,Apartamento no 1º andar com 3 quartos,https://cilar.com.br/alugar/apartamento-no-1-a...,"Avenida Paraná, 111 - Cabral - Curitiba","[Características, do, imóvel, Área, Total, 403...","AluguelR$ 9.000,00","Condominio R$ 2.348,64","IPTU R$ 814,00",Apartamento mobiliado contendo: 1 suíte com cl...,"Elevador, Garagem Coberta","Andar: 01, Ano de construção: 1999, Aptos por ..."
2,Cilar,2024-03-17,Apartamento no 1º andar com 4 quartos,https://cilar.com.br/alugar/apartamento-no-1-a...,"Rua Deputado Joaquim José Pedrosa, 618 - Cabra...","[Características, do, imóvel, Área, Total, 316...","AluguelR$ 5.000,00","Condominio R$ 1.926,04","IPTU R$ 453,00",3 salas com teto rebaixado sendo 1 com sacada ...,"Salão de festas, Playground, Quadra de esporte...","Água: sim, Andar: 01, Ano de construção: 1979,..."
3,Cilar,2024-03-17,Apartamento no 11º andar com 4 quartos,https://cilar.com.br/alugar/apartamento-no-11-...,"Avenida Sete de Setembro, 4699 - Batel - Curitiba","[Características, do, imóvel, Área, Total, 227...","AluguelR$ 4.000,00","Condominio R$ 1.350,00","IPTU R$ 445,00",1 sala para 2 ambientes; 1 sala de jantar; 1 l...,"Academia, Salão de festas, Churrasqueira, Quad...","Andar: 11, Ano de construção: 1989, Aptos por ..."
4,Cilar,2024-03-17,Apartamento com 3 quartos,https://cilar.com.br/alugar/apartamento-com-3-...,"Avenida República Argentina, 1690 - Portão - C...","[Características, do, imóvel, Área, Total, 169...","AluguelR$ 3.900,00","Condominio R$ 804,60","IPTU R$ 195,00","1 cozinha com armário, pia tampo granito, 1 me...",Garagem Coberta,"Ano de construção: 1986, Aptos por andar: 4, Á..."
...,...,...,...,...,...,...,...,...,...,...,...,...
482,Cilar,2024-03-17,Apartamento no 4º andar com 2 quartos,https://cilar.com.br/alugar/apartamento-no-4-a...,"R HOWELL LEWIS FRY, 341 - CACHOEIRA - Curitiba","[Características, do, imóvel, Área, Total, 52,...","AluguelR$ 850,00","Condominio R$ 320,00","IPTU R$ 42,25",Belo apartamento situado em uma área estratégi...,"Salão de festas, Playground, Portaria 24h, Gar...","Andar: 4, Aptos por andar: 4, Área privativa: ..."
483,Cilar,2024-03-17,Apartamento com 2 quartos,https://cilar.com.br/alugar/apartamento-com-2-...,"R Atilio Borio, 139 - Cristo Rei - Curitiba","[Características, do, imóvel, Área, Total, 52,...","AluguelR$ 850,00","Condominio R$ 465,00","IPTU R$ 52,99",Imediações: RODOFERROVIARIA Detalhes: 02 DOR...,Elevador,"Ano de construção: 33 Ano(s), Área privativa: ..."
484,Cilar,2024-03-17,Apartamento com 3 quartos,https://cilar.com.br/alugar/apartamento-com-3-...,"R LUIZ RONALDO CANALLI, 3294 - CAMPO COMPRIDO ...","[Características, do, imóvel, Área, Total, 55,...","AluguelR$ 800,00","Condominio R$ 520,00","IPTU R$ 20,65",Pontos de Referências: Próximo ao Supermercado...,"Playground, Portaria 24h, Garagem Coberta","Aptos por andar: 4, Área privativa: 49.64, Áre..."
485,Cilar,2024-03-17,Apartamento no 5º andar com 1 quarto,https://cilar.com.br/alugar/apartamento-no-5-a...,"Rua Cruz Machado, 311 - Centro - Curitiba","[Características, do, imóvel, Área, Total, 48,...","AluguelR$ 800,00","Condominio R$ 382,00","IPTU R$ 51,19",Apartamento para locação 1 quarto no Centro. ...,NaN,"Andar: 5, Área privativa: 48, Área total: 48, ..."


# Tratando Dados

In [71]:
##### TRATAMENTO INICIAL DOS DADOS #####

df['detalhes'] = df['detalhes'].apply(lambda x: x if isinstance(x,float) else ' '.join(x).replace('Características do imóvel ', ''))

df['iptu'] = df[['condominio','iptu']].apply(lambda x: 
                                            x['iptu'] if (isinstance(x['condominio'],float)) else 
                                            x['condominio'] if ('IPTU' in x['condominio']) else x['iptu'], axis = 1)

df['condominio'] = df[['condominio','iptu']].apply(lambda x: 
                                            x['condominio'] if (isinstance(x['condominio'],float)) else 
                                            np.nan if ('IPTU' in x['condominio']) else x['condominio'], axis = 1)

df['aluguel'] = df['aluguel'].apply(lambda x: x if isinstance(x,float) else int(x.replace('Aluguel','').replace('R$ ','').replace(',00','').replace('.','').split(',')[0]))
df['condominio'] = df['condominio'].apply(lambda x: x if isinstance(x, float) else float(x.replace('Condominio  ','').replace('R$ ', '').replace('.','').replace(',','.')))
df['iptu'] = df['iptu'].apply(lambda x: x if isinstance(x, float) else float(x.replace('IPTU  ','').replace('R$ ','').replace('.',',').replace(',','.')))
df['seguro_incendio'] = 0

def extrai_valores_string(string,substring):

    # Padronizar a expressão regular para encontrar a área total
    padrao = f'{substring} (\d+)'

    # Encontrar a área total usando regex
    area_total = re.search(padrao, string)

    if area_total:
        # Extrair o valor numérico da área total
        valor_area = area_total.group(1)
        
        # Remover vírgulas e converter para float
        valor_area = int(valor_area.replace(',', '.'))
        
    else:
        valor_area = 0
    
    return valor_area

#### FEATURES #####

## Detalhes do imóvel
df['area'] = df['detalhes'].apply(lambda x: x if isinstance(x,float) else extrai_valores_string(x,'Área Total'))
df['quartos'] = df['detalhes'].apply(lambda x: x if isinstance(x,float) else extrai_valores_string(x,'Quartos'))
df['suites'] = df['detalhes'].apply(lambda x: x if isinstance(x,float) else extrai_valores_string(x,'Suítes'))
df['banheiros'] = df['detalhes'].apply(lambda x: x if isinstance(x,float) else extrai_valores_string(x,'Banheiros'))
df['andar'] = df['detalhes'].apply(lambda x: x if isinstance(x,float) else extrai_valores_string(x,'Andar'))
df['vagas_garagem'] = df['mais_detalhes_imovel'].apply(lambda x: 0 if pd.isnull(x) else extrai_valores_string(x,'Vagas de garagem:'))

# Localidade
df['bairro'] = df['endereco'].apply(lambda x: x if isinstance(x,float) else x.split(' - ')[-2])
df['cidade'] = df['endereco'].apply(lambda x: x if isinstance(x,float) else x.split(' - ')[-1])

# Atributos do imóvel e condomínio
df['mobiliado'] = df['catacteristicas_imovel'].apply(lambda x: np.nan if isinstance(x,float) else 'Sim' if 'mobiliado' in unidecode(x.lower()) else 'Não')
df['piscina'] = df['detalhes_condominio'].apply(lambda x: np.nan if isinstance(x,float) else 'Sim' if 'piscina' in unidecode(x.lower()) else 'Não')
df['academia'] = df['detalhes_condominio'].apply(lambda x: np.nan if isinstance(x,float) else 'Sim' if 'academia' in unidecode(x.lower()) else 'Não')
df['sacada'] = df['catacteristicas_imovel'].apply(lambda x: np.nan if isinstance(x,float) else 'Sim' if 'sacada' in unidecode(x.lower()) else 'Não')
df['churrasqueira'] = df['detalhes_condominio'].apply(lambda x: np.nan if isinstance(x,float) else 'Sim' if 'churrasqueira' in unidecode(x.lower()) else 'Não')
df['salao_de_festas'] = df['detalhes_condominio'].apply(lambda x: np.nan if isinstance(x,float) else 'Sim' if 'salao de festa' in unidecode(x.lower()) else 'Não')

#### TRATAMENTO FINAL DE DADOS ####

## Preenchendo valores nulos
# df[['aluguel','condominio','seguro_incendio','iptu']] = df[['aluguel','condominio','seguro_incendio','iptu']].fillna(0)
# df['cidade'] = df['cidade'].fillna('Curitiba')
# df[['area','quartos','suites','banheiros','vagas_garagem']] = df[['area','quartos','suites','banheiros','vagas_garagem']].fillna(0)

## Transformando dtypes
df['data_coleta'] = pd.to_datetime(df['data_coleta'])
df['cidade'] = df['cidade'].apply(lambda x: x if isinstance(x,float) else unidecode(x.capitalize())).astype('category')
df['bairro'] = df['bairro'].apply(lambda x: x if isinstance(x,float) else unidecode(x.capitalize())).astype('category')
df[['aluguel','condominio','seguro_incendio','iptu']] = df[['aluguel','condominio','seguro_incendio','iptu']].astype('float64')

In [72]:
df.isna().sum()

site                       0
data_coleta                0
titulo                     0
link                       0
endereco                   0
detalhes                   0
aluguel                    0
condominio                29
iptu                      76
catacteristicas_imovel     1
detalhes_condominio       61
mais_detalhes_imovel       0
seguro_incendio            0
area                       0
quartos                    0
suites                     0
banheiros                  0
andar                      0
vagas_garagem              0
bairro                     0
cidade                     0
mobiliado                  1
piscina                   61
academia                  61
sacada                     1
churrasqueira             61
salao_de_festas           61
dtype: int64

# Criação de Features

In [64]:
df

,site,data_coleta,titulo,link,endereco,detalhes,aluguel,condominio,iptu,catacteristicas_imovel,...,andar,vagas_garagem,bairro,cidade,mobiliado,piscina,academia,sacada,churrasqueira,salao_de_festas
0,Cilar,2024-03-17,Apartamento no 15º andar com 3 quartos,https://cilar.com.br/alugar/apartamento-no-15-...,"Rua Professor Pedro Viriato Parigot Souza, 306...","Área Total 421,62m2 Quartos 3 Suítes 3 Andar 15",13000.0,3189.70,874.00,1ª LOCAÇÃO Apartamento de alto padrão contend...,...,15,0,Ecoville,Curitiba,Não,Sim,Sim,Sim,Não,Sim
1,Cilar,2024-03-17,Apartamento no 1º andar com 3 quartos,https://cilar.com.br/alugar/apartamento-no-1-a...,"Avenida Paraná, 111 - Cabral - Curitiba","Área Total 403,08m2 Quartos 3 Banheiros 1 Suít...",9000.0,2348.64,814.00,Apartamento mobiliado contendo: 1 suíte com cl...,...,1,0,Cabral,Curitiba,Sim,Não,Não,Sim,Não,Não
2,Cilar,2024-03-17,Apartamento no 1º andar com 4 quartos,https://cilar.com.br/alugar/apartamento-no-1-a...,"Rua Deputado Joaquim José Pedrosa, 618 - Cabra...","Área Total 316,01m2 Quartos 4 Banheiros 1 Suít...",5000.0,1926.04,453.00,3 salas com teto rebaixado sendo 1 com sacada ...,...,1,0,Cabral,Curitiba,Não,Não,Não,Sim,Não,Sim
3,Cilar,2024-03-17,Apartamento no 11º andar com 4 quartos,https://cilar.com.br/alugar/apartamento-no-11-...,"Avenida Sete de Setembro, 4699 - Batel - Curitiba","Área Total 227,00m2 Quartos 4 Banheiros 1 Suít...",4000.0,1350.00,445.00,1 sala para 2 ambientes; 1 sala de jantar; 1 l...,...,11,0,Batel,Curitiba,Não,Não,Sim,Não,Sim,Sim
4,Cilar,2024-03-17,Apartamento com 3 quartos,https://cilar.com.br/alugar/apartamento-com-3-...,"Avenida República Argentina, 1690 - Portão - C...","Área Total 169,62m2 Quartos 3 Banheiros 1 Suít...",3900.0,804.60,195.00,"1 cozinha com armário, pia tampo granito, 1 me...",...,0,0,Portao,Curitiba,Não,Não,Não,Não,Não,Não
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
482,Cilar,2024-03-17,Apartamento no 4º andar com 2 quartos,https://cilar.com.br/alugar/apartamento-no-4-a...,"R HOWELL LEWIS FRY, 341 - CACHOEIRA - Curitiba","Área Total 52,82m2 Quartos 2 Banheiros 1 Andar 4",850.0,320.00,42.25,Belo apartamento situado em uma área estratégi...,...,4,0,Cachoeira,Curitiba,Não,Não,Não,Não,Não,Sim
483,Cilar,2024-03-17,Apartamento com 2 quartos,https://cilar.com.br/alugar/apartamento-com-2-...,"R Atilio Borio, 139 - Cristo Rei - Curitiba","Área Total 52,00m2 Quartos 2 Banheiros 1",850.0,465.00,52.99,Imediações: RODOFERROVIARIA Detalhes: 02 DOR...,...,0,0,Cristo rei,Curitiba,Não,Não,Não,Não,Não,Não
484,Cilar,2024-03-17,Apartamento com 3 quartos,https://cilar.com.br/alugar/apartamento-com-3-...,"R LUIZ RONALDO CANALLI, 3294 - CAMPO COMPRIDO ...","Área Total 55,44m2 Quartos 3 Banheiros 1",800.0,520.00,20.65,Pontos de Referências: Próximo ao Supermercado...,...,0,0,Campo comprido,Curitiba,Não,Não,Não,Não,Não,Não
485,Cilar,2024-03-17,Apartamento no 5º andar com 1 quarto,https://cilar.com.br/alugar/apartamento-no-5-a...,"Rua Cruz Machado, 311 - Centro - Curitiba","Área Total 48,00m2 Quartos 1 Banheiros 1 Andar 5",800.0,382.00,51.19,Apartamento para locação 1 quarto no Centro. ...,...,5,0,Centro,Curitiba,Não,NaN,NaN,Não,NaN,NaN


# Save

In [65]:
columns_selected = [
 'site',
    'titulo',
    'link',
    'data_coleta',
    'endereco',
    'catacteristicas_imovel',
    'detalhes_condominio',
    'mais_detalhes_imovel',
    'bairro',
    'cidade',
    'aluguel',
    'condominio',
    'seguro_incendio',
    'iptu',
    'area',
    'quartos',
    'suites',
    'banheiros',
    'vagas_garagem',
    'mobiliado',
    'piscina',
    'academia',
    'sacada',
    'churrasqueira',
    'salao_de_festas'
]

In [66]:
BUCKET_NAME = 'busca-apartamentos-trusted'
FILE_NAME = f'2024-03-17 - apartamentos - cilar.csv'
TEMP_FILE = 'local.csv'
RAW_PATH = '/tmp/{}'.format(TEMP_FILE)

storage_client = storage.Client()
bucket = storage_client.get_bucket(BUCKET_NAME)

df[columns_selected].to_csv(RAW_PATH, index=False)
blob = bucket.blob(FILE_NAME)
blob.upload_from_filename(RAW_PATH)